In [3]:
from pathlib import Path
import os
import pandas as pd
if not Path('./jw_utils').exists():
    !git clone https://github.com/JonWinkelman/jw_utils.git
if not Path('orthofinder_utils').exists():
    !git clone https://github.com/JonWinkelman/orthofinder_utils.git
from jw_utils import parse_fasta as pfa
from jw_utils import hmmer_utils as hu
from jw_utils import alignment_utils2 as au
from jw_utils import itol_annotations as ia
from jw_utils import ncbi_utils as nu

from Bio import Phylo
from Bio import SeqIO
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import plotly.io as pio
#import make_queries as mq
import random

import subprocess
from concurrent.futures import ThreadPoolExecutor
from jw_utils import make_bac120_tree as mbt
from orthofinder_utils import dash_ortho_parser_d as dop

In [4]:
# Paths 
RESULTS = Path('./results')
DATA=Path('./data')
ASSETS=Path('./assets')
BAC120_PROFILES = DATA / 'bac120_hmm_profiles'
PROTEOME_DIR = DATA / 'Proteomes'
REFERENCE_PROTEOME_PATH = PROTEOME_DIR / 'GCA_000014625.1.faa'
ITOL_ANNOTATION_DIR = RESULTS / 'trees_itol_annotations'
for dir in [RESULTS,DATA,ASSETS, ITOL_ANNOTATION_DIR]:
    dir = Path(dir)
    dir.mkdir(exist_ok=True, parents=True)

### Build bac120 species tree

In [5]:
ab_pres_df = mbt.make_bac120_tree(BAC120_PROFILES, 
                                  PROTEOME_DIR, 
                                  proteome_suffix='.faa', 
                                  output_tree='results/bac120_tree/bac120_tree.nwk',)

2025-04-14 15:26:36,355 - INFO - Concatenating HMM profiles from data/bac120_hmm_profiles to /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm
2025-04-14 15:26:36,460 - INFO - Running command: hmmpress /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm


concatenating HMM profiles into one file and then pressing it into binary...
All .HMM files from 'data/bac120_hmm_profiles' concatenated into '/Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm'.


2025-04-14 15:26:36,911 - INFO - hmmpress output:
Working...    done.
Pressed and indexed 120 HMMs (120 names and 120 accessions).
Models pressed into binary file:   /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm.h3m
SSI index for binary model file:   /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm.h3i
Profiles (MSV part) pressed into:  /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm.h3f
Profiles (remainder) pressed into: /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/mapping_PA14_20480_to_Pseudomonas/results/bac120_tree/bac120hmm_db/concat_bac120_profiles.hmm.h3p



searching each proteome with each HMM using hmmsearch...
aggregating best hit from each proteome into dataframe...
Extracting protein sequences for each HMM from each proteome and writing FASTA files...
5.15% finished.
10.3% finished.
15.15% finished.
20.3% finished.
25.15% finished.
30.3% finished.
35.15% finished.
40.0% finished.
45.15% finished.
50.0% finished.
55.15% finished.
60.3% finished.
65.15% finished.
70.3% finished.
75.15% finished.
80.3% finished.
85.15% finished.
90.3% finished.
95.15% finished.
for each hmm, aligning all extracted proteins with that hmm
Aligning PF01025.23...
Aligning TIGR01171.1...
Aligning TIGR01029.1...
Aligning TIGR01953.1...
Aligning TIGR02386.1...
Aligning TIGR00459.1...
Aligning TIGR00088.1...
Aligning TIGR03632.1...
Aligning TIGR01032.1...
Aligning TIGR01951.1...
Aligning TIGR00029.2...
Aligning TIGR00392.1...
Aligning TIGR00442.1...
Aligning TIGR00810.1...
Aligning TIGR00168.2...
Aligning TIGR00663.1...
Aligning TIGR00396.1...
Aligning TIGR0359

,PF01025.23,TIGR01171.1,TIGR01029.1,TIGR01953.1,TIGR02386.1,TIGR00459.1,TIGR00088.1,TIGR03632.1,TIGR01032.1,TIGR01951.1,...,TIGR00166.1,TIGR03723.1,TIGR01079.1,TIGR01164.1,TIGR00631.1,TIGR01044.1,TIGR00468.1,TIGR00138.1,TIGR00615.1,TIGR02273.1
GCF_001721805.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_000759535.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_019334485.1,True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_900475285.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_014268595.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCF_007833495.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_014522265.1,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_009740185.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
GCF_014076455.1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


#### make itol annotation file to relable species tree names

In [5]:
#### make itol annotation file
df = nu.make_summary_df('./data/summaries.json')
acc_2_name_d = df['organism_name'].to_dict()
relable_dict = {}
for acc, org_name in acc_2_name_d.items():
    new_name = f" {'_'.join(org_name.split(' ')[:2])}_{acc}"
    relable_dict[acc] = new_name

In [6]:
BAC120_TREE_FP = Path('./results/bac120_tree/bac120_tree.nwk')
bac120_tree = Phylo.read(str(BAC120_TREE_FP), format='newick')
itol_relable_species_tree_fp = ITOL_ANNOTATION_DIR / 'itol_relable_species_tree.txt'
ia.relabel_itol_treeleafs(bac120_tree, relable_dict, itol_relable_species_tree_fp)

### make itol annotation file to label presence/absence of bphP, algB, algW putative orthologs

In [7]:
REFERENCE_PROTEOME_ACC='GCA_000014625.1'
df_fp = 'data/GCA_000014625.1__v__GCF_000014625.1_full.csv'
gene_prot_d = pd.read_csv(df_fp).set_index('GCA_gene_ID')['GCA_protein_ID'].to_dict()
query_gene_id_d = {'DimA':'PA14_20480','bphP':'PA14_10700', 'algB':'PA14_72380', 'algW':'PA14_57760'}
name_prot_d = {name:gene_prot_d[gene_id] for name, gene_id in query_gene_id_d.items()}

color_d = {'bphP':'#05BE78','algB':'#C06000', 'algW':'#FF991C'}
rb_result_df_d = {}
for name, prot_id in name_prot_d.items():
    if name != 'DimA':
        rbresults_path = f'./results/rblast_results_{REFERENCE_PROTEOME_ACC}_{prot_id}/recip_blast_results_orgnames.csv'
        rb_result_df = pd.read_csv(rbresults_path).set_index('target_proteome')
        rb_result_df_d[name] = rb_result_df
    
        data_lst_of_ds = []
        for acc, row in rb_result_df.iterrows():
            if row['recip_best_hit']:
                data_lst_of_ds.append({'node': acc, 'label': f'{name}_present', 'color': color_d[name], 'shape': '1'})
            if not row['recip_best_hit']:
                data_lst_of_ds.append({'node': acc, 'label': f'{name}_absent', 'color': '#D7D5D2', 'shape': '1'})
                
        colorstrip_itol_fp = ITOL_ANNOTATION_DIR / f'itol_colorstrip_{name}_presence.txt'
        ia.make_itol_colorstrip_dataset(colorstrip_itol_fp, data_lst_of_ds, dataset_label=f'{name}_presence',
                                     strip_width=50,legend_title=f'{name}_presence', color_branches=0)
    

### make itol annotation file to label P. aeruginosa either clinical or environmental

In [8]:
from jw_utils import plotly_preferences as pp
data_lst_of_ds = []
metadata_df = pd.read_csv('data//PA_strains.csv', index_col=0)
for acc,row in metadata_df.iterrows():
    if row['Type']=='clinical':
        data_lst_of_ds.append({'node': acc, 'label': 'clinical', 'color': '#FF0000', 'shape': '1'})
    else:
        data_lst_of_ds.append({'node': acc, 'label': 'environmental', 'color': '#A0A0A0', 'shape': '1'})
        
clinical_environmental_colorstrip_itol_fp = ITOL_ANNOTATION_DIR / 'itol_colorstrip_clinical-vs-environmental.txt'
ia.make_itol_colorstrip_dataset(clinical_environmental_colorstrip_itol_fp, data_lst_of_ds, dataset_label='clinical-vs-environmental',
                             strip_width=50,legend_title='clinical-vs-environmental', color_branches=0)

### Make colorstrip itol annotation for psuedomonas genera of interest

In [9]:
genera_of_interest = ['chlororaphis','stutzeri', 'fluorescens', 'syringae', 'putida','aeruginosa']
itol_colorstrip_list2 = []
genera_of_interest = ['Pseudomonas', 'Metapseudomonas', 'Halopseudomonas', 'Geopseudomonas',
                      'Ectopseudomonas', 'Phytopseudomonas', 'Aquipseudomonas', 'Candidatus', 'Stutzerimonas']
for acc, org_name in acc_2_name_d.items():
    genus = org_name.split(' ')[0]
    species = ' '.join(org_name.split(' ')[:2])
    if species == 'Pseudomonas aeruginosa':
        itol_colorstrip_list2.append({'node':acc, 'label':genus, 'color':'#330000', 'shape':'1'})
    elif genus in genera_of_interest:
        itol_colorstrip_list2.append({'node':acc, 'label':genus, 'color':'#3399FF', 'shape':'1'})
    else:
        itol_colorstrip_list2.append({'node':acc, 'label':genus, 'color':'#C0C0C0', 'shape':'1'})


itol_colorstrip_fp = 'results/trees_itol_annotations/itol_genus_colorstrip.txt'
ia.make_itol_colorstrip_dataset(itol_colorstrip_fp, itol_colorstrip_list2, dataset_label='Psuedomonas_genera', legend_title='Psuedomonas_genera', 
                               strip_width=50, color_branches=1)



In [10]:
dimA_rblast_df = pd.read_csv('results/rblast_results_GCA_000014625.1_ABJ12623.1/recip_blast_results_orgnames.csv', index_col=0)

# add manual hits to dict  -- Manually detected dimA
acc_prot_id_d = {
                "GCF_024168455.1"  : "WP_253559225.1", 
                "GCF_029350995.1"  : "WP_077934638.1", 
                "GCF_023241665.1"  : "WP_268261674.1", 
                "GCA_902497625.1"  : "WP_150579783.1", 
                "GCF_028657095.1"  : "WP_273924074.1", 
                "GCF_044900625.1"  : "WP_404697551.1", 
                "GCF_045062265.1"  : "WP_087273078.1", 
                "GCF_001698815.1"  : "WP_065832684.1", 
                "GCF_014357575.1"  : "WP_187522871.1", 
                "GCF_022790535.1" : "WP_243246698.1", 
                "GCF_900103875.1" : "WP_090187081.1",
                } 
# add rblast hits to dict
for acc, row in dimA_rblast_df.iterrows():
    if row['recip_best_hit']:
        acc_prot_id_d[acc] = row['f_bbhit']

data_lst_of_ds = []
all_dimA_hits = list(acc_prot_id_d.keys())
for acc, row in dimA_rblast_df.iterrows():
    if acc in all_dimA_hits:
        data_lst_of_ds.append({'node': acc, 'label': 'present', 'color': '#AD07E3', 'shape': '1'})
    else:
        data_lst_of_ds.append({'node': acc, 'label': 'present', 'color': '#D7D5D2', 'shape': '1'})
        
dimA_colorstrip_itol_fp = ITOL_ANNOTATION_DIR / 'itol_colorstrip_DimA.txt'
ia.make_itol_colorstrip_dataset(dimA_colorstrip_itol_fp, data_lst_of_ds, dataset_label='dimA',
                             strip_width=50,legend_title='dimA', color_branches=0)

### make itol dataset to annotate trees with absence/presence of WTF MOTIF

In [11]:
def extract_prot_seqs(proteome_dir, acc_prot_id_d):
    """"""
    seq_d = {}
    for acc, prot_id  in acc_prot_id_d.items():
        fp = Path(proteome_dir) / f'{acc}.faa'
        f_seq_d = pfa.get_seq_dict(fp)
        seq = f_seq_d[prot_id]
        full_id = f'{acc}_{prot_id}'
        seq_d[full_id] = seq
    return seq_d

seq_d  = extract_prot_seqs(PROTEOME_DIR, acc_prot_id_d)

In [12]:
import re
pattern = r"[WL][TVI]F$"
motif_d = {}
for full_id, seq in seq_d.items():
    m = re.search(pattern, seq[-3:]) 
    if m:
        motif_d[full_id[:15]] = [True, m[0]]
    else:
        motif_d[full_id[:15]] = [False, '']

In [13]:
data_lst_of_ds = []
for acc, d_lst  in motif_d.items():
    if d_lst[0]:
        data_lst_of_ds.append({'node': acc, 'label': 'present', 'color': '#B6202F', 'shape': '2'})
    else:
        data_lst_of_ds.append({'node': acc, 'label': 'absent', 'color': '#E2EDF3', 'shape': '2'})
        
motif_colorstrip_itol_fp = ITOL_ANNOTATION_DIR / 'itol_colorstrip_motif.txt'
ia.make_itol_colorstrip_dataset(motif_colorstrip_itol_fp, data_lst_of_ds, dataset_label='motif_present',
                             strip_width=50,legend_title='motif_present', color_branches=0)